## **Import Dependencies**

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import os
from glob import glob
import pylandstats as pls
import warnings
import rasterio as rio
import rasterio.mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
import argparse
from tqdm import tqdm

# Suppress warnings to keep output clean
warnings.filterwarnings('ignore')

In [15]:
import psutil

# Get virtual memory stats
mem = psutil.virtual_memory()

# Print in human-readable format
print(f"Total RAM: {mem.total / (1024 ** 3):.2f} GB")
print(f"Available RAM: {mem.available / (1024 ** 3):.2f} GB")
print(f"Used RAM: {mem.used / (1024 ** 3):.2f} GB")
print(f"RAM Usage: {mem.percent}%")

Total RAM: 187.59 GB
Available RAM: 118.35 GB
Used RAM: 67.83 GB
RAM Usage: 36.9%


## Define File and Directory Paths

In [16]:
# Main directory containing raw raster and vector datasets
MAIN_DATA_DIR = r'/beegfs/halder/DATA'

# GitHub-linked project directory where processed data and results are stored
PROJECT_DATA_DIR = r'/beegfs/halder/GITHUB/Landscape-Analysis/data'

# Temporary directory used for storing intermediate files
TEMP_DIR = os.path.join(PROJECT_DATA_DIR, 'TEMP')

## Extract the Crop Mask Combining 5 Years of Crop Type Data

In [18]:
# # Read the crop file paths
# crop_file_paths = [os.path.join(MAIN_DATA_DIR, 'DE_Crop_Types_2017_2021', f'DE_Crop_Type_{year}.tif') for year in range(2017, 2022)]

# # Define the crop code
# crop_code = {
#     'wheat': 1110,
#     'barley': 1120,
#     'maize': 1130,
#     'rapeseed': 1430,
#     'sugarbeet': 1320
# } 

# target_crs = CRS.from_epsg(25832)

# # Use first file to define target transform, shape
# with rio.open(crop_file_paths[0]) as src:
#     transform, width, height = calculate_default_transform(
#         src.crs, target_crs, src.width, src.height, *src.bounds
#     )
#     profile = src.profile
#     profile.update({
#         'crs': target_crs,
#         'transform': transform,
#         'width': width,
#         'height': height,
#         'dtype': 'uint8',
#         'count': 1,
#         'nodata': 0
#     })

# # Initialize combined masks
# combined_masks = {
#     crop: np.zeros((height, width), dtype=np.uint8) for crop in crop_code
# }

# # Loop through and reproject, then apply masks
# for fp in tqdm(crop_file_paths):
#     with rio.open(fp) as src:
#         reprojected = np.zeros((height, width), dtype=src.dtypes[0])
#         reproject(
#             source=src.read(1),
#             destination=reprojected,
#             src_transform=src.transform,
#             src_crs=src.crs,
#             dst_transform=transform,
#             dst_crs=target_crs,
#             resampling=Resampling.nearest
#         )

#         for crop, code in crop_code.items():
#             mask = (reprojected == code).astype(np.uint8)
#             combined_masks[crop] += mask


# # Create output directory
# OUT_DIR = os.path.join(MAIN_DATA_DIR, 'DE_Crop_Types_2017_2021')
# os.makedirs(OUT_DIR, exist_ok=True)

# # Write each combined mask to disk
# for crop, mask_array in combined_masks.items():
#     out_path = os.path.join(OUT_DIR, f"{crop}_mask_combined.tif")
#     with rio.open(out_path, 'w', **profile) as dst:
#         dst.write(mask_array, 1)